<a href="https://colab.research.google.com/github/Carvas91/PyTroch/blob/main/manual_split_ann_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import torch.nn as nn
import seaborn as sns

In [2]:
iris = sns.load_dataset('iris')

data = torch.tensor(iris[iris.columns[0:4]].values).float()

labels = torch.zeros(len(data), dtype=torch.long)
labels[iris.species == 'versicolor'] = 1
labels[iris.species == 'virginica'] = 2

In [11]:
labels

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2])

In [30]:
propTraining = .8
nTraining = int(len(labels)*propTraining)


traintestBool = np.zeros(len(labels),dtype=bool)

items2use4train = np.random.choice(range(len(labels)), nTraining, replace=False)

traintestBool[items2use4train] = True

traintestBool

array([ True,  True, False,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
       False,  True,  True,  True, False,  True, False,  True, False,
        True,  True,  True,  True,  True, False,  True,  True,  True,
       False,  True, False,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True, False, False,  True, False,  True,
        True, False,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True, False,  True,  True,  True,  True, False,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
       False, False,  True,  True, False,  True,  True, False, False,
        True,  True,

In [31]:
len(data[traintestBool])

120

In [32]:
len(data[~traintestBool])

30

In [33]:
torch.mean(labels.float())

tensor(1.)

In [34]:
ANNIris = nn.Sequential(
    nn.Linear(4,64),
    nn.ReLU(),
    nn.Linear(64,64),
    nn.ReLU(),
    nn.Linear(64,3)
)

lossfun = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(ANNIris.parameters(), lr=0.01)

In [35]:
print(data.shape)

print(data[traintestBool].shape)

print(data[~traintestBool].shape)

torch.Size([150, 4])
torch.Size([120, 4])
torch.Size([30, 4])


In [36]:
epochs =1000

losses = torch.zeros(epochs)

ongoingacc = []


for i in range(epochs):
  yHat = ANNIris(data[traintestBool])

  ongoingacc.append(100* torch.mean(
      (torch.argmax(yHat, axis=1) == labels[traintestBool]).float()
  ))

  loss = lossfun(yHat, labels[traintestBool])
  losses[i] = loss

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()



In [37]:
trainpreds = ANNIris(data[traintestBool])

trainacc = torch.mean((torch.argmax(trainpreds,axis=1) == labels[traintestBool]).float()) * 100



testpreds = ANNIris(data[~traintestBool])

testacc = torch.mean(
    (torch.argmax(testpreds, axis=1) == labels[~traintestBool]).float()
)*100


print('Train acc %g%%' %trainacc)

print('Test acc %g%%' %testacc)

Train acc 97.5%
Test acc 100%
